In [1]:
'''
IBN GRANITE MODELS
https://github.com/ibm-granite/granite-code-models


To use any of our models, pick an appropriate model_path from:
ibm-granite/granite-3b-code-base
ibm-granite/granite-3b-code-instruct
ibm-granite/granite-8b-code-base
ibm-granite/granite-8b-code-instruct
ibm-granite/granite-20b-code-base
ibm-granite/granite-20b-code-instruct
ibm-granite/granite-34b-code-base
ibm-granite/granite-34b-code-instruct
'''
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

device = "cuda" # or "cpu"
model_path = "ibm-granite/granite-3b-code-instruct" # pick anyone from above list

tokenizer = AutoTokenizer.from_pretrained(model_path)
# Config added from repo to handle larger text inputs
config = AutoConfig.from_pretrained(model_path)
# config.max_seq_len = 4096
# config.max_answer_len= 1024
config.max_new_tokens = 250
config.max_length = 250

# drop device_map if running on CPU
model_code = AutoModelForCausalLM.from_pretrained(model_path
                                            #  , device_map=device
                                             , config = config)
model_code.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at ibm-granite/granite-3b-code-instruct were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_proj.bias', 'model.layers.11.mlp.up_proj.bias', 'model.layers.12.mlp.down_proj.bias', 'model.layers.12.mlp.gate_proj.bias', 'model.layers.12.mlp.up_proj.bias', 'model.layers.13.mlp.down_proj.bias', 'model.layers.13.mlp.gate_proj.bias', 'model.layers.13.mlp.up_proj.bias', 'model.layers.14.mlp.down_proj.bias', 'model.layers.14.mlp.gate_proj.bias', 'model.layers.14.mlp.up_proj.bias', 'model.layers.15.mlp.down_proj.bias', 'model.layers.15.mlp.gate_proj.bias', 'model.layers.15.m

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2560, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (up_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (down_proj): Linear(in_features=10240, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
 

In [2]:
def get_information(model, question, device):
    input_tokens = tokenizer(question, return_tensors="pt")

    # transfer tokenized inputs to the device
    # for i in input_tokens:
    #     input_tokens[i] = input_tokens[i].to(device)

    # generate output tokens
    output = model.generate(**input_tokens)
    # decode output tokens into text
    output = tokenizer.batch_decode(output)
    return output

In [3]:
test_text = "def generate():"
test_out = get_information(model_code, test_text, device)
# loop over the batch to print, in this example the batch size is 1
for i in test_out:
    print(i)

/Users/jorgepinzon/opt/anaconda3/envs/morgage/lib/python3.12/site-packages/transformers/generation/utils.py:1256: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [ ]:
base = '''what does this function do {}'''
example1 = base.format('''
even= []
uneven=[]
for i in INPUT:
  if i %2 ==0:
    even.append(i)
  else:
    uneven.append(i)
org = uneven + even
print(org)''')
# loop over the batch to print, in this example the batch size is 1
for i in get_information(model_code, example1, device):
    print(i)


In [ ]:
example2 = base.format('''
final = []
for item in INPUT:
  if item %2 ==0:
    final.insert(len(final)+1, item)
  else:
    final.insert(0, item)
print(final)''')
# loop over the batch to print, in this example the batch size is 1
for i in get_information(model_code, example2, device):
    print(i)

In [ ]:
example3 = base.format('''import pandas as pd
def create_df():
    data = {'state': ['Ohio','Ohio','Ohio','Nevada','Nevada'],
           'year': [2000,2001,2002,2001,2002],
           'pop': [1.5,1.7,3.6,2.4,2.9]}
    df = pd.DataFrame(data)
    return df

### We'll create three dataframes for an example
for i in range(3):
    exec(f'df_{i} = create_df()')''')
for i in get_information(model_code, example3, device):
    print(i)